In [ ]:
from torchvision import models
import torch
import pickle as pk # to save list as file
import gc # garbage collector, can help you free memory
!pip install import_ipynb
import import_ipynb # tool allow you to import from .ipynb files. Usualy import only from .py files.
from os.path import exists



In [ ]:
from create_dataset import DicomDataset, DicomTestDataset # my dataset class
from create_dataset import train_metadata_df # import dataset csv

In [ ]:
# Creating instance of DicomDataset
from torchvision.transforms import ToTensor, ToPILImage


training_data = DicomDataset(
    merged_df=train_metadata_df,
    image_dir="train",
#    train=True,
#    download=True,
    preload = False,
    transform=ToTensor()
)



testing_data = DicomTestDataset(
    image_dir="test",
#    train=True,
#    download=True,
    preload = False,
    transform=ToTensor()
)

In [ ]:
%whos # checking memory usage

In [ ]:
# torch can use CUDA, this line check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model_dict = {"alexnet":models.alexnet(pretrained=True),"resnet50":models.resnet50(pretrained=True), 
"densenet161":models.densenet161(pretrained=True), 
"wide_resnet50_2":models.wide_resnet50_2(pretrained=True),
"vgg13":models.vgg13(pretrained=True)
}

def calc_features(data_set , model_dict, name_part = '', recalc = False ):
    
    for model_name in model_dict.keys():
        file_name = name_part + model_name + ".fea"
        if (not exists(file_name) ) or (recalc):
            
            torch.cuda.empty_cache()
            model = model_dict[model_name]
            model.eval()
            model.to(device)
            
            ds_res= []
            print("start ",model_name)
            for image,lable in data_set:
                #print(image)
                with torch.no_grad():
                    im_res = model(torch.unsqueeze(image,0).to(device))
                    
                ds_res.append(im_res.to('cpu'))
                del im_res
                torch.cuda.empty_cache()
                gc.collect()
                
            #save as binary file
            pk.dump(ds_res, open(file_name, 'wb'))
            print(f"Saved features from {model_name} model into {file_name} ")
            del ds_res
    


In [ ]:

calc_features(training_data, model_dict)
calc_features(testing_data, model_dict,name_part = "test_", recalc = True)
testing_data[0]

In [ ]:
from create_dataset import DicomDataset, DicomTestDataset # my dataset class
gc.collect()
hsv_training_data = DicomDataset(
    merged_df=train_metadata_df,
    image_dir="train",
    bone = False,
#    train=True,
#    download=True,
    preload = False,
    transform=ToTensor()
)

hsv_testing_data = DicomTestDataset(
    image_dir="test",
    bone = False,
#    train=True,
#    download=True,
    preload = False,
    transform=ToTensor()
)

calc_features(hsv_training_data, model_dict,name_part = "hsv_")
gc.collect()
calc_features(hsv_testing_data, model_dict,name_part = "test_hsv_")
gc.collect()
